In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import gensim
import spacy 
import nltk 



In [14]:

df = pd.read_csv("season.csv")
df.head(16)

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/14/screen_name,user_mentions/15/id_str,user_mentions/15/name,user_mentions/15/screen_name,user_mentions/16/id_str,user_mentions/16/name,user_mentions/16/screen_name,user_mentions/17/id_str,user_mentions/17/name,user_mentions/17/screen_name
0,1474707073342394370,2021-12-25T11:42:19.000Z,24,???? ????? ???? ???? ?? #????????_????_???? \n...,????????_????_????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1474688020070780937,2021-12-25T10:26:37.000Z,27,???? ????? ?? #????_?????? \n??? ???? .. ?????...,????_??????,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1474537577676001283,2021-12-25T00:28:48.000Z,60,???? ????? ???????? .. ?? ??? ?????? ??\n?????...,????????_????_????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1474510457771675649,2021-12-24T22:41:02.000Z,68,?? ?????? ???? ??????? \n?????? ?????? ???? ??...,????????_????_????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1474508006465236995,2021-12-24T22:31:18.000Z,35,???? ????????? ?? ???? .. ???????? ?????? ????...,????_?????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1474501540572393473,2021-12-24T22:05:36.000Z,61,???? ?? ??????? ??????? \n??????? ?? ??????? ?...,??_?????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1474501141287165953,2021-12-24T22:04:01.000Z,26,?????? ????? ?????? \n??????? ???? ???? ??????...,??_?????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1474500686473691137,2021-12-24T22:02:13.000Z,47,??? ???? ?? ???? ??\n????? ????? ????? ????? ?...,????????_????_????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1474456118017486849,2021-12-24T19:05:07.000Z,47,???? ???????? ??????? .. ????? ??????? ????? ?...,??????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1474454487687319552,2021-12-24T18:58:38.000Z,62,??? ???? ???? ?????? ???? ??????? ???\n??? ???...,????????_????_????,????_??????,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# adapted from https://github.com/bakrianoo/aravec
# function to clean and normalize text 
def clean_text(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    
    return text


In [ ]:
# adapted from https://github.com/bakrianoo/aravec


# Load AraVec model from gensim    
model = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")

# export to word2vec format for spacy model
model.wv.save_word2vec_format("./spacyModel/aravec.txt")
# run the following command gzip ./spacyModel/aravec.txt   
# run the following command python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz
 
# load spacy model 
nlp = spacy.load("./spacy.aravec.model/")